In [22]:
import glob
import os
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
ProgressBar().register()

In [36]:
# considerando que as as bases de 2005 até 2020 são fixas
# vamos criar um único dataframe com esse período
df = dd.read_csv(['2005_2020.dask/*.part', '2021_em_diante.dask/*.part'], sep=";")
print(f'O dataframe tem {len(df)} registros')
df.info()

[########################################] | 100% Completed | 35.29 s
O dataframe tem 62570765 registros
<class 'dask.dataframe.core.DataFrame'>
Columns: 9 entries, Unnamed: 0 to NR_COTST
dtypes: object(2), float64(5), int64(2)

In [35]:
# transforma o campo CO_PRD e DT_COMPTC
df['CNPJ_FUNDO'] = df['CNPJ_FUNDO'].str.replace('.', '')
df['CNPJ_FUNDO'] = df['CNPJ_FUNDO'].str.replace('/', '')
df['CNPJ_FUNDO'] = df['CNPJ_FUNDO'].str.replace('-', '')
df['CNPJ_FUNDO'] = df['CNPJ_FUNDO'].str.zfill(14)
df['DT_COMPTC'] = pd.to_datetime(df['DT_COMPTC'], errors='coerce').dt.strftime('%Y-%m-%d')
df

# formata o campo CO_PRD com o cnpj
print('Formatando campo CO_PRD')
df = df.dropna(subset=['CNPJ_FUNDO'])
df['CO_PRD'] = df['CNPJ_FUNDO'].str.replace(".", "")
df['CO_PRD'] = df['CO_PRD'].str.replace("-", "")
df['CO_PRD'] = df['CO_PRD'].str.replace("/", "")
df['CO_PRD'] = df['CO_PRD'].str.zfill(14)

# converte o dataframe dask para pandas
print('Convertendo o df dask para pandas')
df = df.compute()

# cria e formata o campo DT_REF, com a data de referência - errors='raise'
print('Formatando campo DT_REF')
df.assign(DT_COMPTC=pd.to_datetime(df['DT_COMPTC'], format='%Y-%m-%d', errors='coerce'))
df = df.dropna(subset=['DT_COMPTC'])
df['DT_REF'] = df['DT_COMPTC']

print(f' Antes de remover duplicados, o dataframe tem {len(df)} registros')
# remove registros duplicados, baseados no CO_PRD e DT_REF
print('Removendo registros duplicados')
df = df.drop_duplicates(subset=['CO_PRD', 'DT_REF'], keep='last')
print(f'Depois de remover duplicados, o dataframe tem {len(df)} registros')

# cria uma nova coluna com o percentual de resgate para o dia
print('Calculando campo PC_RESG')
df['PC_RESG'] = (df['RESG_DIA'] / df.groupby(['CO_PRD'])['VL_PATRIM_LIQ'].shift(1))

[########################################] | 100% Completed | 193.56 s
[########################################] | 100% Completed | 196.53 s
[########################################] | 100% Completed | 336.40 ms
[################                        ] | 42% Completed | 53.85 ss


KeyboardInterrupt: 